In [265]:
import pandas as pd
import altair as alt
import altair_latimes as lat

In [266]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [267]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import

In [268]:
df = pd.read_csv("../data/processed/district-level-residential-use.csv", parse_dates=["reporting_month"])

In [269]:
baselines_df = pd.read_csv("../data/raw/uw-2020-baseline-values.csv")

### Clean

Clean up baselines column names as we did in the previous notebook

In [270]:
baselines_df.columns = baselines_df.columns.str.replace(" ","_").str.replace("-","_").str.replace(r'[#,@,&,(,)]', '', regex=True).str.lower()

In [271]:
baselines_df.supplier_name = baselines_df.supplier_name.str.replace("  ", " ")

In [272]:
baselines_trimmed_df = baselines_df[[
    'supplier_name', 
    'public_water_system_id', 
    'month', 
    #'original_units',
    # 'total_potable_production_original_units',
    # 'potable_commercial_agriculture_original_units',
    'total_potable_production_minus_ag_gallons', 
    # 'staff_notes'
]].copy()

Get rid of some unnecessary columns

In [273]:
trim_df = df[[
    'supplier_name', 
    'public_water_system_id', 
    'reporting_month', 
    'county',
    'hydrologic_region', 
    # 'climate_zone', 
    'total_population_served',
    'calculated_total_potable_water_production_gallons_ag_excluded',
    # 'calculated_r_gpcd', 
    # 'pop_weighted_rgpcd', 
    # 'month'
]].copy()

### Merge

Filter df to July 2021 and later

In [307]:
filter_df = trim_df[trim_df.reporting_month >= "2021-06-15"].copy()

Make a month column to merge with baselines

In [308]:
filter_df["month"] = filter_df["reporting_month"].dt.month

In [309]:
merge_df = pd.merge(
    filter_df, 
    baselines_trimmed_df, 
    how="left", 
    on=["supplier_name", "public_water_system_id", "month"]
).rename(
    columns={
        "calculated_total_potable_water_production_gallons_ag_excluded": "total_gallons_current",
        "total_potable_production_minus_ag_gallons": "total_gallons_baseline"
    }
).drop("month", axis=1)

### Calculate percent changes by month

In [310]:
def pct_change(new, old):
    return (new - old) / old

By district

In [311]:
merge_df["gallons_pct_change"] = merge_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

By region

In [312]:
regions_df = merge_df.groupby(["hydrologic_region", "reporting_month"])[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

In [313]:
regions_df["gallons_pct_change"] = regions_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [317]:
regions_df[(regions_df.hydrologic_region == "South Coast")&(regions_df.reporting_month>="2021-07-15")]

,hydrologic_region,reporting_month,total_gallons_current,total_gallons_baseline,gallons_pct_change
71,South Coast,2021-07-15,98840547682.225,98231994555.628,0.006
72,South Coast,2021-08-15,99534276774.714,101674411259.822,-0.021
73,South Coast,2021-09-15,91814533261.595,94897438912.390,-0.032
74,South Coast,2021-10-15,80644440759.798,90861162006.982,-0.112
75,South Coast,2021-11-15,76030984586.050,74797486800.110,0.016
76,South Coast,2021-12-15,60816667800.139,74019981000.887,-0.178
77,South Coast,2022-01-15,61447605343.398,59998217456.723,0.024
78,South Coast,2022-02-15,63965355832.248,63116859652.232,0.013
79,South Coast,2022-03-15,72247093556.118,56449514888.188,0.280


Statewide

In [318]:
statewide_df = merge_df.groupby(["hydrologic_region"])[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

In [319]:
statewide_df["gallons_pct_change"] = statewide_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

### Calculate cumulative savings since July 2021

Drop suppliers with missing data

In [320]:
counts = merge_df.groupby("supplier_name")["reporting_month"].count().reset_index()

In [321]:
max_count = max(counts.reporting_month)

In [322]:
suppliers_with_complete_data = list(counts[counts.reporting_month == max_count].supplier_name)

In [344]:
complete_data = merge_df[merge_df.supplier_name.isin(suppliers_with_complete_data)].copy()

By district

In [345]:
district_cumulative_savings_df = complete_data.groupby(["supplier_name","hydrologic_region"])[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

In [346]:
district_cumulative_savings_df["cumulative_pct_change"] = complete_data.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [347]:
district_cumulative_savings_df

,supplier_name,hydrologic_region,total_gallons_current,total_gallons_baseline,cumulative_pct_change
0,Adelanto City of,South Lahontan,1223905830.448,1263786920.057,-0.110
1,Alameda County Water District,San Francisco Bay,11331000000.000,12150200000.000,-0.031
2,Alco Water Service,Central Coast,1083142000.000,1114537000.000,-0.066
3,Alhambra City of,South Coast,2520769577.593,2583428317.598,0.066
4,Amador Water Agency,San Joaquin River,1029370000.000,1061830000.000,0.079
...,...,...,...,...,...
347,Westminster City of,South Coast,2946898688.700,2957069114.882,-0.122
348,"Windsor, Town of",North Coast,798709352.799,1029971067.399,-0.011
349,Woodland City of,Sacramento River,2672098974.000,2836923315.000,-0.020
350,Yorba Linda Water District,South Coast,5329889412.330,5538838107.570,0.133


By region

In [348]:
regions_cumulative_savings_df = district_cumulative_savings_df.groupby(
    ["hydrologic_region"]
)[
    ["total_gallons_current","total_gallons_baseline","supplier_name"]
].agg(
    {"total_gallons_current":"sum","total_gallons_baseline":"sum","supplier_name":"size"}
).reset_index().rename(
    columns={"supplier_name": "total_reports"}
)

In [349]:
regions_cumulative_savings_df["cumulative_pct_change"] = regions_cumulative_savings_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [350]:
regions_cumulative_savings_df

,hydrologic_region,total_gallons_current,total_gallons_baseline,total_reports,cumulative_pct_change
0,Central Coast,37252334773.580,38397097468.589,25,-0.030
1,Colorado River,49285274361.661,48672475614.415,9,0.013
2,North Coast,11299322827.318,13107817474.506,15,-0.138
3,North Lahontan,3356302137.000,3579635596.000,4,-0.062
4,Sacramento River,149466369876.494,156500560874.047,41,-0.045
5,San Francisco Bay,196853066534.531,215987302424.975,42,-0.089
6,San Joaquin River,76789875686.876,78671372670.795,22,-0.024
7,South Coast,751253092705.079,755545242320.251,153,-0.006
8,South Lahontan,34199157481.151,34504752311.420,13,-0.009
9,Tulare Lake,114404152725.043,117113914983.965,28,-0.023


Statewide

In [351]:
complete_data["state"] = "California"

In [352]:
statewide_cumulative_savings_df = district_cumulative_savings_df.groupby(
    ["state"]
)[
    ["total_gallons_current","total_gallons_baseline", "supplier_name"]
].agg({
    "total_gallons_current": "sum",
    "total_gallons_baseline": "sum",
    "supplier_name": "size"
}).reset_index().rename(
    columns={"supplier_name": "total_reports"}
)

In [353]:
#statewide_cumulative_savings_df["total_reports"] = statewide_cumulative_savings_df["total_reports"]

In [354]:
statewide_cumulative_savings_df["cumulative_pct_change"] = statewide_cumulative_savings_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [355]:
statewide_cumulative_savings_df

,state,total_gallons_current,total_gallons_baseline,total_reports,cumulative_pct_change
0,California,1424158949108.733,1462080171738.963,3520,-0.026


### Charts

In [300]:
alt.Chart(
    regions_df[regions_df.hydrologic_region=="South Coast"]
).mark_bar().encode(
    x="reporting_month:O",
    y="gallons_pct_change:Q",
    color=alt.condition(
        alt.datum.gallons_pct_change > 0,
        alt.value("#e6ae56"),  # The positive color
        alt.value("#83c6e0")  # The negative color
    ),
    tooltip=["gallons_pct_change"]
).properties(title="Monthly water conservation in the South Coast", width=600)

alt.Chart(...)

### Export

Monthly

In [301]:
statewide_df.to_csv("../data/processed/monthly-conservation/statewide-conservation-monthly.csv", index=False)

In [302]:
regions_df.to_csv("../data/processed/monthly-conservation/regional-conservation-monthly.csv", index=False)

In [303]:
merge_df.to_csv("../data/processed/monthly-conservation/district-level-conservation-monthly.csv", index=False)

Cumulative

In [304]:
statewide_cumulative_savings_df.to_csv("../data/processed/cumulative-conservation/statewide-conservation-cumulative.csv", index=False)

In [305]:
regions_cumulative_savings_df.to_csv("../data/processed/cumulative-conservation/monthly-conservation-cumulative.csv", index=False)

In [306]:
district_cumulative_savings_df.to_csv("../data/processed/cumulative-conservation/district-level-conservation-cumulative.csv", index=False)